In [18]:
!pip3 install pandas
!pip3 install plotly
!pip3 install numpy
!pip3 install seaborn
!pip3 install matplotlib.pyplot
!pip3 install mpl_toolkits
!pip3 install sklearn
!pip3 install statsmodels
!pip3 install random
!pip3 install time


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement mpl_toolkits (from versions: none)
ERROR: No matching distribution found for mpl_toolkits

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
  Usin


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [47]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.compose import make_column_transformer

import time
import statsmodels.api as sm
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
from random import shuffle, seed

In [20]:
data = pd.read_csv('data/nyc-waste-data.csv')
data.head()

,Material,Aggregate Percent,Refuse Percent,MGP Percent,Paper Percent,Organic Percent,Material Group,DSNY Diversion Summary Category,Location
0,Newspaper,0.019,0.011,0.005,0.098,0.000,Paper,Designated Paper,Citywide
1,Plain OCC/Kraft Paper,0.056,0.012,0.010,0.464,0.002,Paper,Designated Paper,Citywide
2,Paper Bags: Kraft Grocery,0.002,0.001,0.000,0.008,0.000,Paper,Designated Paper,Citywide
3,Paper Bags: Non-food Retail,0.002,0.002,0.000,0.008,0.000,Paper,Designated Paper,Citywide
4,Paper Bags: Fast Food Bags,0.001,0.002,0.000,0.001,0.000,Paper,Designated Paper,Citywide


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832 entries, 0 to 1831
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Material                         1832 non-null   object 
 1   Aggregate Percent                1832 non-null   float64
 2   Refuse Percent                   1832 non-null   float64
 3   MGP Percent                      1603 non-null   float64
 4   Paper Percent                    1603 non-null   float64
 5   Organic Percent                  1145 non-null   float64
 6   Material Group                   1832 non-null   object 
 7   DSNY Diversion Summary Category  1832 non-null   object 
 8   Location                         1832 non-null   object 
dtypes: float64(5), object(4)
memory usage: 128.9+ KB


In [22]:
# understand general distribution and categorization of materials 
material_fig = px.histogram(data, x='Material Group', title='Material Categorization')
material_fig.show()

In [23]:
# apply basic regression model 
# target: material 
# numerical features: aggregate, refuse, MGP, paper, organic percentage 

X = data.drop('Material Group', axis = 1)
y = data["Material Group"]


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

# num_data = data.iloc[:, 1:7]
# num_data.head()

In [25]:
numerical_cols=['Aggregate Percent', 'Refuse Percent', 'MGP Percent', 'Paper Percent', 'Organic Percent']
categorical_cols=['DSNY Diversion Summary Category', 'Location', 'Material']

numerical_transformer = SimpleImputer(strategy='constant')
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
preprocessor = ColumnTransformer(transformers=[('num', numerical_transformer, numerical_cols), ('cat', categorical_transformer, categorical_cols)])


In [26]:
baseline = data['Material Group'].value_counts(normalize = True)
print("baseline for all materials: ", baseline)

baseline for all materials:  Material Group
Plastic                     0.711790
Paper                       0.069869
Special Waste               0.056769
Organic                     0.052402
Metal                       0.043668
Glass                       0.026201
E-Waste                     0.021834
C&D                         0.013100
Miscellaneous Inorganics    0.004367
Name: proportion, dtype: float64


In [27]:
model_comparisons_dict = {'Model':[], # 'Logistic Regression', 'KNN', 'Decision Tree', 'SVM'
        'Train Time':[],
        'Train Accuracy':[],
        'Test Accuracy':[]
       }
model_comparisons = pd.DataFrame(model_comparisons_dict)

In [28]:
# Model 1: Logistic Regression attempt 
model = LogisticRegression(max_iter=10000, random_state=42)
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('logreg', model)
])

log_reg_start_time = time.time()
pipe.fit(X_train, y_train)
log_reg_end_time = time.time()

log_reg_train_time = log_reg_end_time - log_reg_start_time
print("Train time for logistic regression: ", log_reg_train_time)

Train time for logistic regression:  0.08336210250854492


In [29]:
preds = pipe.predict(X_test)
# Evaluate the accuracy 
log_reg_test_accuracy = accuracy_score(y_test, preds)
log_reg_train_accuracy = accuracy_score(y_train, pipe.predict(X_train))

print("accuracy score (train): ", log_reg_train_accuracy)
print("accuracy score (test): ", log_reg_test_accuracy)

model_comparisons.loc[len(model_comparisons.index)] = ['Logistic Regression', log_reg_train_time, log_reg_train_accuracy, log_reg_test_accuracy] 

accuracy score (train):  0.9906396255850234
accuracy score (test):  0.9545454545454546


In [32]:
# Model 2: KNN with n = 5 default

knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier(n_neighbors=5))])

# Fitting the pipeline and measuring time 
knn_start_time = time.time()
knn_pipeline.fit(X_train, y_train)
knn_end_time = time.time()

# Collect metrics 
knn_train_time = knn_end_time - knn_start_time; 
print("Train time for KNN: ", knn_train_time)

preds = pipe.predict(X_test)

Train time for KNN:  0.024687767028808594


In [34]:
# Evaluate the accuracy 
knn_train_accuracy = accuracy_score(y_train, pipe.predict(X_train))
knn_test_accuracy = accuracy_score(y_test, preds)
print("knn accuracy score (train): ", knn_train_accuracy)
print("knn accuracy score (test): ", knn_test_accuracy)
model_comparisons.loc[len(model_comparisons.index)] = ['KNN (n = 5)', knn_train_time, knn_train_accuracy, knn_test_accuracy] 

knn accuracy score (train):  0.9906396255850234
knn accuracy score (test):  0.9545454545454546


In [36]:
# Model 3: Decision Tree with max depth = 3
dt_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=3))
])
# Fitting the pipeline and measuring time 
dt_start_time = time.time()
dt_pipeline.fit(X_train, y_train)
dt_end_time = time.time()

# Collect metrics 
dt_train_time = dt_end_time - dt_start_time; 
dt_accuracy_test = dt_pipeline.score(X_test, y_test)
dt_accuracy_train = dt_pipeline.score(X_train, y_train)

print("Decision Tree Train Time: ", dt_train_time)
print("Decision Tree Train Accuracy: ", dt_accuracy_train)
print("Decision Tree Test Accuracy: ", dt_accuracy_test)

# Add results to table
model_comparisons.loc[len(model_comparisons.index)] = ['Decision Tree (depth = 3)', dt_train_time, dt_accuracy_train,dt_accuracy_test]


Decision Tree Train Time:  0.024069547653198242
Decision Tree Train Accuracy:  0.8190327613104524
Decision Tree Test Accuracy:  0.8036363636363636


In [37]:
# Model 4: SVM 
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear'))
])

# Fitting the pipeline  and measuring time 
svm_start_time = time.time()
svm_pipeline.fit(X_train, y_train)
svm_end_time = time.time()

# Collect metrics 
svm_train_time = svm_end_time - svm_start_time; 
svm_accuracy_test = svm_pipeline.score(X_test, y_test)
svm_accuracy_train = svm_pipeline.score(X_train, y_train)

print("SVM Train Time: ", svm_train_time)
print("SVM Train Accuracy: ", svm_accuracy_train)
print("SVM Test Accuracy: ", svm_accuracy_test)

# Add results to table
model_comparisons.loc[len(model_comparisons.index)] = ['SVM (linear)', svm_train_time, svm_accuracy_train, svm_accuracy_test]
                                                       

SVM Train Time:  0.2765331268310547
SVM Train Accuracy:  1.0
SVM Test Accuracy:  1.0


In [43]:
# SVM Model is likely overfitted as seen by accuracy = 1.0. 
# Can use cross validation to better assess this model 

# Create a 5 fold cross-validation object
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(kernel='linear'))
])

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
accuracy_scores = cross_val_score(svm_pipeline, X, y, cv=kfold, scoring='accuracy')
print("Accuracy scores for each fold:", accuracy_scores)

mean_accuracy = accuracy_scores.mean()
std_accuracy = accuracy_scores.std()

print(f"Mean accuracy: {mean_accuracy:.4f}")

Accuracy scores for each fold: [1. 1. 1. 1. 1.]
Mean accuracy: 1.0000


In [53]:
# Hmm looks like this either may be a very simplistic classification for the SVM model or a hyperparameter
# needs to be modified. Use GridSearchCV to explore parameters 

# GridSearchCV for SVM pipeline 
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svm', SVC(kernel='poly'))
])

param_grid = {
    'svm__C': [0.1, 1, 10],  # SVM regularization parameter
    'svm__kernel': ['linear', 'rbf'],  # SVM kernel types
    'svm__gamma': [0.1, 1, 10]  # SVM kernel coefficient for RBF kernel
}

grid_search = GridSearchCV(svm_pipeline, param_grid, cv=5, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X, y)

# Get the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best accuracy:", grid_search.best_score_)

Best parameters: {'svm__C': 1, 'svm__gamma': 0.1, 'svm__kernel': 'linear'}
Best accuracy: 1.0


In [ ]:
# Looks like that did not help - it is possible to conclude that this classification problem is too simple for this 
# model type! 

# Let's look at all other models and plot accuracies now 

In [55]:
model_comparisons.head()

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.083362,0.990640,0.954545
1,KNN (n = 5),0.024688,0.990640,0.954545
2,Decision Tree (depth = 3),0.024070,0.819033,0.803636
3,SVM (linear),0.276533,1.000000,1.000000


In [57]:
# Plot model against test accuracy 
fig = px.scatter(model_comparisons, x='Model', y='Test Accuracy',
                 title='Model Performance: Test Accuracy')
fig.show()

fig2 = px.scatter(model_comparisons, x='Train Time', y='Test Accuracy', text='Model',
                 title='Model Performance: Train Time vs Test Accuracy')
fig2.show()

In [58]:
# this gives a decent sense of which models are high-performant in terms of time and accuracy. We can make some
# high-level conclusions from this: 
# - An SVM model seems to be highly accurate but also requires the most amount of time 
# - A KNN with n=5 takes less time and is just as accurate as the Logistic Regression model
# - The Decision Tree with a max depth of 3 does not have as high an accuracy as the other models. However, more
# depths may be explored and could yield different results for this model type

In [ ]:
# Next steps: we have done a bulk of the number crunching here, but in the final report we will 
# explore even higher levels of accuracy and hyperparameter optimization. For example, KNN model at other N values. 
# We can also explore other measurements of accuracy and model quality. Then, we will recommend a model for
# the business use case of NYC waste classification!